In [1]:
import os
from pathlib import Path
import mediapy
import matplotlib.pyplot as plt

working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)

from gpudrive.env.dataset import SceneDataLoader
from gpudrive.env.config import EnvConfig
from gpudrive.env.env_torch import GPUDriveTorchEnv
from gpudrive.visualize.utils import img_from_fig

import logging
logging.basicConfig(level=logging.INFO)

%load_ext autoreload
%autoreload 2

/workspace/gpudrive/.venv/lib/python3.11/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


### Dataset

In [2]:
train_loader = SceneDataLoader(
    root="data/processed/examples",
    batch_size=4, # Number of worlds
    dataset_size=1000,
    sample_with_replacement=False,
    shuffle=False,
)

### Model


In [ ]:
from examples.experimental.eval_utils import load_policy, rollout

policy = load_policy(
    path_to_cpt='examples/experimental/models',  # ???
    model_name='',
    device='cpu'
)

/workspace/gpudrive/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: 'examples/experimental/models/.pt'

In [14]:
# Check that the model weights are not random
# for name, param in policy.state_dict().items():
#     print(f"{name} - Mean: {param.mean():.4f}, Std: {param.std():.4f}")

In [ ]:
policy

### GPUDriveTorchEnv

In [ ]:
env = GPUDriveTorchEnv(
    config=EnvConfig(),
    data_loader=train_loader,
    max_cont_agents=64, 
    device="cpu",
)

print(env.data_batch)

obs = env.reset()[env.cont_agent_mask]

print(f'observation_space: {env.observation_space}')
print(f'obs shape: {obs.shape}')
print(f'obs dtype: {obs.dtype} \n')

print(f'action_space: {env.action_space}')

plt.hist(obs.flatten());

In [ ]:
# Show simulator to make sure we're at the same state
env.vis.figsize = (5, 5)
sim_states = env.vis.plot_simulator_state(
    env_indices=[0],
    zoom_radius=100,
    time_steps=[0],
)

sim_states[0]

In [ ]:
( 
 goal_achieved_count,
 frac_goal_achieved,
 collided_count,
 frac_collided,
 off_road_count,
 frac_off_road,
 not_goal_nor_crash_count,
 frac_not_goal_nor_crash_per_scene,
 controlled_agents_per_scene,
 sim_state_frames,
 agent_positions,
 episode_lengths
) = rollout(
    env=env, 
    policy=policy, 
    device='cpu', 
    render_sim_state=True,
    zoom_radius=100,
    deterministic=True,
)

print(f'\n Results: \n')
print(f'Goal achieved: {frac_goal_achieved}')
print(f'Collided: {frac_collided}')
print(f'Off road: {frac_off_road}')
print(f'Not goal nor crashed: {frac_not_goal_nor_crash_per_scene}')

In [ ]:
# Show rollout videos
mediapy.show_videos(sim_state_frames, fps=15, codec='gif')